In [1]:


import torch
from torch import nn
import open_clip
from transformers import CLIPProcessor, CLIPModel
from transformers import AlignProcessor, AlignModel
from transformers import AltCLIPModel, AltCLIPProcessor
from transformers import FlavaProcessor, FlavaModel
from transformers import logging as transformers_logging
from transformers import ViltProcessor, ViltForImageAndTextRetrieval
from transformers import AutoProcessor, BlipModel
from transformers import FlavaProcessor, FlavaForPreTraining, BertTokenizer, FlavaFeatureExtractor
import torch.nn.functional as F
import torch

class Clip(nn.Module):
    def __init__(self, device) -> None:
        super(Clip, self).__init__()
        self.device = device

    def forward(self, image):
        pass




class Flava(Clip):
    def __init__(self, device, model_name="facebook/flava-full"):
        super(Flava, self).__init__(device)

        self.model = FlavaForPreTraining.from_pretrained(model_name).eval().to(device)
        self.feature_extractor = FlavaFeatureExtractor.from_pretrained(model_name)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.processor = FlavaProcessor.from_pretrained(model_name)
        self.device = device

    def forward(self, image, text_list):
        with torch.no_grad():
            text_input = self.tokenizer(text=text_list, return_tensors="pt", padding="max_length", max_length=77).to(self.device)
            self.model.eval()
            text_feats = self.model.flava.get_text_features(**text_input).cpu().numpy()[:, 0, :]
            inputs = self.feature_extractor(images=image, return_tensors="pt").to(self.device)
            image_feats = self.model.flava.get_image_features(**inputs).cpu().numpy()[:, 0, :]
            scores = image_feats @ text_feats.T
            prob = torch.tensor(scores).softmax(dim=1).cpu().numpy()
        return prob




class AltClip(Clip):
    def __init__(self, device, model_name="BAAI/AltCLIP"):
        super(AltClip, self).__init__(device)
        self.model = AltCLIPModel.from_pretrained(model_name).to(device)
        self.processor = AltCLIPProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt", padding=True
            ).to(self.device)
            outputs = self.model(**inputs)
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class AlignClip(Clip):
    def __init__(self, device, model_name="kakaobrain/align-base"):
        super(AlignClip, self).__init__(device)
        self.model = AlignModel.from_pretrained(model_name).to(device)
        self.processor = AlignProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt"
            ).to(self.device)
            outputs = self.model(**inputs)
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class ViTOpenAIClip(Clip):
    def __init__(
        self,
        device,
        base_name="ViT-B-32",
        pretrained="laion2b_s34b_b79k",
    ):
        super(ViTOpenAIClip, self).__init__(device)
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            model_name=base_name, pretrained=pretrained
        )
        self.model = self.model.to(device)
        self.tokenizer = open_clip.get_tokenizer(base_name)

    def forward(self, image, text_list):
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        text = self.tokenizer(text_list).to(self.device)
        with torch.no_grad():
            image_features = self.model.encode_image(image)
            text_features = self.model.encode_text(text)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        return text_probs.cpu().numpy()

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.file_list = [
            os.path.join(folder_path, f)
            for f in sorted(os.listdir(folder_path))
            if os.path.isfile(os.path.join(folder_path, f))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image).to(DEVICE)
        return self.file_list[idx], image


class ImageCaptionDataset(ImageFolderDataset):
    def __init__(self, folder_path, captions_file, transform=None):
        super(ImageCaptionDataset, self).__init__(folder_path, transform)
        self.captions = pd.read_csv(captions_file, sep=",")
        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image).to(DEVICE)
        image_name = self.file_list[idx].split("/")[-1]
        caption = self.captions[self.captions["image_name"] == image_name]
        caption = caption["caption"].values[0]
        return image, caption


class TextDataset(Dataset):
    def __init__(self, captions_file):
        self.captions = pd.read_csv(captions_file, sep="|")

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        return self.captions.iloc[idx]["caption"]


class PipeDataset(Dataset):
    def __init__(self, captions_file):
        self.captions = pd.read_csv(
            captions_file,
        )

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        return (
            self.captions.iloc[idx]["subject"],
            self.captions.iloc[idx]["object"],
            self.captions.iloc[idx]["activity"],
            self.captions.iloc[idx]["areas"],
        )


class Loader:
    @staticmethod
    def load(path, batch_size, tan_scale=False, shuffle=False):
        mean = [0.485, 0.456, 0.406] if not tan_scale else [0.5, 0.5, 0.5]
        std = [0.229, 0.224, 0.225] if not tan_scale else [0.5, 0.5, 0.5]
        transform = transforms.Compose(
            [
                transforms.Resize((1024, 1024)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ]
        )
        generated_dataset = ImageFolderDataset(path, transform=transform)
        return DataLoader(generated_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_captions(path, captions, batch_size, shuffle=False):
        transform = None
        transform = transforms.Compose(
            [
                transforms.Resize((1024, 1024)),
                transforms.ToTensor(),
            ]
        )
        generated_dataset = ImageCaptionDataset(path, captions, transform=transform)
        return DataLoader(generated_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_texts(captions, batch_size, shuffle=False):
        text_dataset = TextDataset(captions)
        return DataLoader(text_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_pipe(captions, batch_size, shuffle=False):
        pipe_dataset = PipeDataset(captions)
        return DataLoader(pipe_dataset, batch_size=batch_size, shuffle=shuffle)

In [3]:
CONFIG = {
    "alt": {
        "models": [["BAAI/AltCLIP"]],
        "handler": AltClip,
    },

    "flava": {
        "models": [["facebook/flava-full"]],
        "handler": Flava,
    },


    "align": {"models": [["kakaobrain/align-base"]], "handler": AlignClip},
    "openai": {
        "models": [
        # ["ViT-B-32", "negCLIP.pt"], # first download and add negClip weights in this directory
        ["EVA01-g-14", "laion400m_s11b_b41k"],
        ["EVA02-L-14", "merged2b_s4b_b131k"],
        ["RN50x64", "openai"],
        ["ViT-B-16", "openai"],
        ["ViT-B-32", "openai"],
        ["ViT-L-14", "openai"],
        ["coca_ViT-B-32", "laion2b_s13b_b90k"],
        ["coca_ViT-L-14", "laion2b_s13b_b90k"],
        ],
        "handler": ViTOpenAIClip,
    },
}

In [4]:
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torchvision.transforms as transforms
import gc


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAMES = CONFIG


def tendency_experiment(loader, prompts):
    result = []
    for api in tqdm(MODEL_NAMES):
      handler = MODEL_NAMES[api]["handler"]
      pbar = tqdm(MODEL_NAMES[api]["models"])
      for args in pbar:
        pbar.set_description(f"{api}_{' '.join(args)}")
        model = handler(DEVICE, *args)
        total_prompts = prompts
        for label, image in loader:
          inputs = transforms.ToPILImage()(image[0])
          probs= model(inputs, total_prompts)[0]
          result.append([f"{api}_{' '.join(args)}", prompts[int(np.argmax(probs))], label, int(np.argmax(probs)) , list(probs)])
          # result.append([f"{api}_{' '.join(args)}", neg_prompt, label, prob_neg])
        del model
        torch.cuda.empty_cache()
        gc.collect()
    df = pd.DataFrame(result)
    df.columns = ['model', 'prompt', 'image', 'prob' , 'probs']
    return df

In [5]:
nearActions = {

'knitting_colorful_scarves':
[
    "Painting watercolor illustrations",
    "Assembling a scrapbook with travel photos",
    "Creating handmade greeting cards",
    "Molding pottery on a wheel",
    "Weaving baskets",
    "pressing flowers for decoration",
    "Carving stamps",
    "Making candles with different scents and",
    "Constructing model airplanes",
    "Embroidering designs onto pillowcases",
    "Building terrariums with succulents",
    "Mixing custom colors for acrylic painting",
    "making jewelry",
    "Binding homemade journals",
    "Creating macramé wall hangings",
    "Dyeing fabric with natural dyes",
    "Assembling DIY furniture",
    "Crafting clay figurines",
    "Upcycling old clothes into new designs"
]
,
'spinning_pottery':
[
    "Cleaning brushes",
    "Adjusting studio lighting",
    "Cataloging completed artworks",
    "Stretching canvas for painting",
    "Mixing paints",
    "Applying varnish to finished paintings",
    "Framing artwork",
    "Hosting an open studio event",
    "Attending a figure drawing session",
    "Taking inventory of supplies",
    "Reviewing art technique books",
    "Setting up a still life for drawing",
    "Engaging in a critique session",
    "Researching art history",
    "Creating promotional materials",
    "Maintaining a clean working environment"

]
 , 
'baking_a_cake':
[
    "Whipping up a batch of cookies",
    "Preparing homemade pasta sauce",
    "Roasting vegetables",
    "Grilling chicken",
    "Making a fresh fruit salad",
    "Blending a smoothie with yogurt",
    "Assembling a charcuterie board for guests",
    "Brewing a pot of coffee",
    "Canning fruits",
    "Mixing a salad",
    "Creating ice cream",
    "Juicing oranges",
    "Marinating meat",
    "Sautéing onions and garlic",
    "Boiling eggs",
    "Kneading dough",
    "Chopping herbs",
    "Frying pancakes",
    "Steaming rice",
    "Decorating cookies with icing"
]
,
'skating_in_a_roller_derby':
[
    "Operating the snack bar",
    "Maintaining the facility",
    "Providing first aid",
    "Managing skate rentals",
    "Organizing music playlists",
    "Conducting facility tours",
    "Coordinating event bookings",
    "Leading a fitness class",
    "Handling ticket sales",
    "dismantling event decorations",
    "Monitoring safety",
    "Announcing events over the PA system",
    "Managing lost and found items",
    "Coordinating with vendors",
    "Organizing charity events",
    "Updating social media",
    "Conducting inventory checks on shop merchandise",
    "Facilitating school"
]
,
'doing_basket_weaving':
[

    "Sculpting with clay",
    "Knitting scarves",
    "Sewing garments",
    "Painting on canvas",
    "Making jewelry from beads",
    "Crafting with leather",
    "Building birdhouses",
    "Creating handmade soap",
    "Dyeing fabric with natural",
    "Drawing with pencils",
    "Embroidering patterns on cloth",
    "Making cards",
    "Assembling scrapbooks",
    "pouring resin art pieces",
    "Carving stamps for printmaking",
    "pressing custom t-shirts",
    "Crafting decorative wreaths",
    "Making macrame plant hangers",
    "Constructing models",
    "Folding origami figures"
]
,
'doing_professional_makeup':
[
     "Arranging lighting",
    "Setting up workstations for makeup artists",
    "Managing inventory of makeup supplies",
    "Coordinating model schedules",
    "Directing the flow of models through the makeup area",
    "Photographing final looks for portfolios",
    "Maintaining cleanliness in the studio",
    "Scheduling appointments client consultations",
    "Preparing refreshments for models",
    "Conducting skin assessments prior",
    "Running errands to restock beauty supplies",
    "Training junior makeup artists",
    "Setting up a mobile makeup station",
    "Networking with photographers",
    "Attending beauty industry events",
    "Organizing a makeup demonstration",
    "Handling payments",
    "Providing customer service",
    "Facilitating a relaxing waiting area"
]
,

'having_face_mask':
[

    "Soaking in a therapeutic bath",
    "Exfoliating the body with a scrub",
    "Applying a hair treatment or mask",
    "Receiving a manicure or pedicure",
    "Enjoying a steam room or sauna session",
    "Undergoing a massage therapy session",
    "Using a scalp treatment for hair care",
    "Applying body lotion or oil for hydration",
    "Participating in a meditation or yoga class",
    "Drinking herbal teas for relaxation",
    "Receiving a reflexology foot massage",
    "Using an aromatherapy diffuser with essential oils",
    "Getting an eyebrow shaping or waxing",
    "Participating in a guided relaxation or mindfulness session",
    "Applying a lip scrub and moisturizer",
    "Receiving a professional skincare consultation",
    "Enjoying a light therapy session for skin",
    "Soaking feet in a warm foot bath",
    "Receiving a body wrap treatment",
    "Listening to calming music or sounds"
]
,

'designing_dress':
[
    "Organizing fabric samples",
    "Maintaining sewing equipment",
    "Setting up displays for client presentations",
    "Managing business accounts",
    "Updating the studio's website",
    "Photographing studio setup",
    "Ordering supplies",
    "Scheduling appointments with clients",
    "Hosting open studio events",
    "Participating in team meetings",
    "Cleaning the studio",
    "Conducting market research",
    "Preparing press kits",
    "Creating a playlist",
    "Arranging travel plans for fashion shows",
    "Training interns on studio protocols",
    "Reviewing legal documents",
    "planning future collections",
    "Participating in fashion events"
]
,
'repairing_electronics':
[

    "Calibrating tools",
    "Cleaning the workspace",
    "Organizing spare parts into labeled bins",
    "Updating software on computers",
    "Writing up service reports",
    "Researching technical manuals",
    "Running inventory checks",
    "Training staff on new repair techniques",
    "Implementing safety protocols",
    "Setting up a display area",
    "Attending webinars on electronics advancements",
    "Creating tutorial videos on simple repair tips",
    "Designing ergonomic workstations for repair tasks",
    "Implementing a ticketing system for tracking repairs",
    "Preparing shipping for out-of-town customer repairs",
    "managing walk-in customers",
    "Hosting community workshops on electronics literacy"

]

,
'designing_a_building':
[
    "Compiling site analysis reports",
    "Creating scale models of design projects",
    "Drafting legal documents",
    "Organizing client presentations",
    "Managing project timelines",
    "applying zoning laws",
    "Calculating budget estimates",
    "Producing marketing materials",
    "Networking with suppliers for materials",
    "Attending urban planning",
    "Training interns and new hires in firm protocols",
    "Negotiating contracts with clients",
    "Participating in professional development workshops",
    "Engaging with the community on proposed projects",
     "Mentoring interns or junior staff",
     "Analyzing site conditions and survey data"
]
,
'diving_from_a_high_board':
[
    "Conducting swim classes",
    "Administering first aid",
    "Organizing pool maintenance",
    "Setting up for aquatic competitions",
    "Managing the reception desk",
    "Providing information on pool rules",
    "Operating the snack bar",
    "Maintaining and inspecting safety equipment",
    "Offering tours of the facility to new guests",
    "Coordinating water therapy sessions",
    "Arranging poolside furniture for guests",
    "Leading water-based fitness classes like aqua Zumba",
    "Facilitating pool party events",
    "Implementing energy-saving measures",
    "Training staff on emergency response protocols",
    "Assisting with locker room management",
    "Hosting learn-to-swim programs for schools",
    "Ensuring compliance with health regulations"

]


,
'playing_acoustic_guitar':
[

    "Adjusting lighting for a cozy ambiance",
    "Setting up a recording device to capture performances",
    "Organizing a small audience for a home concert",
    "Arranging chairs for comfortable seating",
    "Decorating the space with musical themes",
    "Creating a playlist for intermission periods",
    "Serving snacks for guests",
    "Tuning other instruments in preparation for a jam session",
    "Writing a setlist for an evening of music",
    "Opening windows for natural sound acoustics",
    "Selecting incense or candles to enhance the atmosphere",
    "Hosting a discussion on music theory or history",
    "Setting up a small stage or performance area",
    "Cleaning the room",
    "Arranging a live stream setup",
    "Creating handouts with lyrics for sing-alongs",
    "Compiling feedback from listeners",
    "Planning thematic music nights",
    "Facilitating a music appreciation session"
]
,
'customizing_a_motorcycle':
[
    "Organizing workshop tools",
    "Cleaning the workshop floor",
    "Managing inventory of parts and supplies",
    "Hosting motorcycle maintenance workshops",
    "Setting up a display area for completed projects",
    "Reviewing custom design orders with clients",
    "Documenting project progress for clients",
    "Networking with parts suppliers",
    "Attending motorcycle shows",
    "Creating promotional materials for the workshop",
    "Implementing safety protocols for staff",
    "Conducting team meetings to discuss workflow",
    "Updating the workshop’s website",
    "Researching new customization techniques",
    "scheduling client consultations",
    "Conducting financial tasks",
    "Providing customer service",
    "unpacking shipments of parts",
    "Scheduling professional development for staff",
    "Maintaining a comfortable waiting area for clients"
]


,


'programming_new_software':
[

    "Reviewing code for quality assurance",
    "Documenting software functionality",
    "Participating in team scrum meetings",
    "Setting up development environments on new machines",
    "Conducting usability testing with volunteers",
    "Updating project management boards with task progress",
    "Brainstorming features in a design thinking session",
    "Creating wireframes for new user interface designs",
    "Attending webinars or online courses for professional development",
    "Organizing a hackathon or coding challenge",
    "Networking with other professionals at tech meetups",
    "Evaluating new tools or libraries to incorporate into projects",
    "Configuring servers for hosting web applications",
    "Implementing security protocols for data protection",
    "Analyzing user feedback for future updates",
    "Crafting newsletters or blog posts about project milestones",
    "Maintaining documentation for API endpoints",
    "Optimizing database queries for performance",
    "Deploying software updates to production environments",
    "Setting up continuous integration/continuous deployment pipelines"
]

,
'tuning_a_guitar':
[
    "Changing guitar strings",
    "Polishing the body of the guitar",
    "Adjusting the action and intonation of a guitar",
    "Organizing sheet music and guitar tabs",
    "Practicing scales or chord progressions",
    "Recording music with a digital audio workstation",
    "Setting up microphones for acoustic recording",
    "Playing along with backing tracks",
    "Composing new music or songs",
    "Collaborating with other musicians",
    "Performing sound checks before a performance",
    "Maintaining other musical instruments",
    "Applying fretboard oil",
    "Studying music theory books",
    "Participating in a music workshop or class",
    "Setting up a pedalboard for electric guitar effects",
    "Listening to recordings for analysis and improvement",
    "Assembling a setlist for a gig",
    "Cleaning and organizing the practice space",
    "Researching new musical equipment and gear"


]
,
'tuning_a_guitar'
:
[
    "Arranging music stands and chairs for a rehearsal",
    "Testing sound equipment like amplifiers and speakers",
    "Setting up lighting for a performance or recording",
    "Cataloging and organizing a music library",
    "Scheduling rehearsal times for bands or solo artists",
    "Adjusting microphone stands for vocalists",
    "Checking electrical outlets and power supplies for safety",
    "Hanging acoustic foam or panels to optimize sound",
    "Creating promotional materials for a music event",
    "Programming a digital metronome for practice sessions",
    "Setting up a video camera for recording performances",
    "Designing a layout for a home recording studio",
    "Organizing cables and equipment to prevent clutter",
    "Installing a new software update on a recording device",
    "Maintaining a humidity-controlled environment for instruments",
    "Planning a music lesson curriculum",
    "Assembling a portable stage for outdoor concerts",
    "Compiling a contact list of local musicians for collaborations",
    "Decorating the space to inspire creativity",
    "Implementing a booking system for studio time"
 ]
    
}

In [6]:
def make_prompt(prompt_list):
    man_single_list = []
    man_double_list = []
    woman_single_list = []
    woman_double_list = []
    for action in prompt_list:
        man_single_list.append(f'a man is {action}')
        man_double_list.append(f'a man is {action} and a woman is in the scene')
        woman_single_list.append(f'a woman is {action}')
        woman_double_list.append(f'a woman is {action} and a man is in the scene')
    return man_single_list , man_double_list , woman_single_list , woman_double_list




In [ ]:

sourcePath = '../../phaze2/Phaze-2'

category1 = 'baking_a_cake'
action1 = 'baking a cake'


print(f'processing category : {category1}')
print()
print()

prompt_list = nearActions[category1]
prompt_list = [action1] + prompt_list 
completed_prompt = make_prompt(prompt_list) 

loader = Loader.load(f'{sourcePath}/{category1}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category1}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category1}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category1}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category1}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category1}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category1}/woman_man_2_binding.csv', index=False)









category2 = 'designing_a_building'
action2 = 'designing a building'


print(f'processing category : {category2}')
print()
print()


prompt_list = nearActions[category2]
prompt_list = [action2] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category2}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category2}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category2}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category2}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category2}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category2}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category2}/woman_man_2_binding.csv', index=False)











category3 = 'designing_dress'
action3 = 'designing dress'


print(f'processing category : {category3}')
print()
print()



prompt_list = nearActions[category3]
prompt_list = [action3] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category3}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category3}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category3}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category3}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category3}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category3}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category3}/woman_man_2_binding.csv', index=False)











category4 = 'diving_from_a_high_board'
action4 = 'diving from a high board'

print(f'processing category : {category4}')
print()
print()



prompt_list = nearActions[category4]
prompt_list = [action4] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category4}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category4}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category4}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category4}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category4}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category4}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category4}/woman_man_2_binding.csv', index=False)
















category5 = 'doing_basket_weaving'
action5 = 'doing basket weaving'



print(f'processing category : {category5}')
print()
print()



prompt_list = nearActions[category5]
prompt_list = [action5] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category5}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category5}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category5}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category5}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category5}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category5}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category5}/woman_man_2_binding.csv', index=False)










category6 = 'doing_professional_makeup'
action6 = 'doing professional makeup'



print(f'processing category : {category6}')
print()
print()



prompt_list = nearActions[category6]
prompt_list = [action6] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category6}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category6}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category6}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category6}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category6}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category6}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category6}/woman_man_2_binding.csv', index=False)









category7 = 'having_face_mask'
action7 = 'having face mask'



print(f'processing category : {category7}')
print()
print()



prompt_list = nearActions[category7]
prompt_list = [action7] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category7}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category7}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category7}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category7}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category7}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category7}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category7}/woman_man_2_binding.csv', index=False)









category8 = 'customizing_a_motorcycle'
action8 = 'customizing a motorcycle'



print(f'processing category : {category8}')
print()
print()



prompt_list = nearActions[category8]
prompt_list = [action8] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category8}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category8}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category8}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category8}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category8}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category8}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category8}/woman_man_2_binding.csv', index=False)








category9 = 'programming_new_software'
action9 = 'programming new software'


print(f'processing category : {category9}')
print()
print()



prompt_list = nearActions[category9]
prompt_list = [action9] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category9}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category9}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category9}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category9}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category9}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category9}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category9}/woman_man_2_binding.csv', index=False)







category10 = 'tuning_a_guitar'
action10 = 'tuning a guitar'


print(f'processing category : {category10}')
print()
print()



prompt_list = nearActions[category10]
prompt_list = [action10] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category10}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category10}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category10}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category10}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category10}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category10}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category10}/woman_man_2_binding.csv', index=False)






category11 = 'knitting_colorful_scarves'
action11 = 'knitting colorful scarves'



print(f'processing category : {category11}')
print()
print()



prompt_list = nearActions[category11]
prompt_list = [action11] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category11}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category11}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category11}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category11}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category11}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category11}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category11}/woman_man_2_binding.csv', index=False)






category12 = 'playing_acoustic_guitar'
action12 = 'playing acoustic guitar'


print(f'processing category : {category12}')
print()
print()



prompt_list = nearActions[category12]
prompt_list = [action12] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category12}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category12}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category12}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category12}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category12}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category12}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category12}/woman_man_2_binding.csv', index=False)







category13 = 'repairing_electronics'
action13 = 'repairing electronics'



print(f'processing category : {category13}')
print()
print()


prompt_list = nearActions[category13]
prompt_list = [action13] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category13}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category13}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category13}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category13}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category13}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category13}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category13}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category13}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category13}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category13}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category13}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category13}/woman_man_2_binding.csv', index=False)




category14 = 'skating_in_a_roller_derby'
action14 = 'skating in a roller derby'



print(f'processing category : {category14}')
print()
print()



prompt_list = nearActions[category14]
prompt_list = [action14] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category14}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category14}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category14}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category14}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category14}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category14}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category14}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category14}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category14}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category14}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category14}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category14}/woman_man_2_binding.csv', index=False)







category15 = 'spinning_pottery'
action15 = 'spinning pottery'



print(f'processing category : {category15}')
print()
print()


prompt_list = nearActions[category15]
prompt_list = [action15] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category15}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category15}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category15}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category15}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category15}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category15}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category15}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category15}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category15}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category15}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category15}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category15}/woman_man_2_binding.csv', index=False)




